# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [13]:
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

keys = get_keys("/Users/tjjj/mia/.secret/yelp_api.json")

client_id = keys['client_id']
api_key = keys['api_key']


In [48]:
import requests

term = 'fish and chips'
location = 'Kent WA'
SEARCH_LIMIT = 5
search_radius = 16093 #about 10 miles from Kent, WA

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'radius': search_radius
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "QKIH0ALmEGsRuFRDRRn9oQ", "alias": "wallys-chowder-house-and-broiler-des-moines", "name": "Wally's Chowder House & Broiler", "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/BCqtc7AoigjHtZrTcoI_jg/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/wallys-chowder-house-and-broiler-des-moines?adjust_creative=2k3unDeFyr0jFUAk_Ochag&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2k3unDeFyr0jFUAk_Ochag", "review_count": 1140, "categories": [{"alias": "seafood", "title": "Seafood"}, {"alias": "fishnchips", "title": "Fish & Chips"}], "rating": 4.5, "coordinates": {"latitude": 47.39945, "longitude": -122.32482}, "transactions": [], "price": "$$", "location": {"address1": "22531 Marine View Dr S", "address2": "", "address3": "", "city": "Des Moines", "zip_code": "98198", "country": "US", "state": "WA", "display_address": ["22531 Marine View Dr S", "Des Moines, WA 98198"]}, "phone": "+12068788140", "displ

In [49]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [50]:
response.json()['total']

65

In [51]:
len(response.json()['businesses'])

5

In [60]:
response.json()['businesses'][0]

{'id': 'QKIH0ALmEGsRuFRDRRn9oQ',
 'alias': 'wallys-chowder-house-and-broiler-des-moines',
 'name': "Wally's Chowder House & Broiler",
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/BCqtc7AoigjHtZrTcoI_jg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/wallys-chowder-house-and-broiler-des-moines?adjust_creative=2k3unDeFyr0jFUAk_Ochag&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=2k3unDeFyr0jFUAk_Ochag',
 'review_count': 1140,
 'categories': [{'alias': 'seafood', 'title': 'Seafood'},
  {'alias': 'fishnchips', 'title': 'Fish & Chips'}],
 'rating': 4.5,
 'coordinates': {'latitude': 47.39945, 'longitude': -122.32482},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '22531 Marine View Dr S',
  'address2': '',
  'address3': '',
  'city': 'Des Moines',
  'zip_code': '98198',
  'country': 'US',
  'state': 'WA',
  'display_address': ['22531 Marine View Dr S', 'Des Moines, WA 98198']},
 'phone': '+12068788140',
 'display_phone': '(206)

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [53]:
response = requests.get(url, headers=headers, params=url_params)

In [65]:
def yelp_api_request(parameters):
    '''Return a list of dicts (businesses) == the number of total results.'''
    counter = 0 
    final_result = []
    
    response = requests.get(url, headers=headers, params=parameters)
    results = response.json()['businesses']
    total_results = response.json()['total']
    len_results = len(results)
    
    for i in results:
        final_result.append(i)
        counter += 1
    
    if counter < total_results:
        yelp_api_request(parameters)
        
    elif counter == total_results:
        pass #helpful!!! 
    
    return final_result

In [66]:
cristian = yelp_api_request(url_params)
len(cristian)

KeyboardInterrupt: 

## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
#Your code here

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
#Your code here

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!